# Efficient Quantum Expectation Values Estimation for Concentrated States

## Summary

Measuring [expectation values](https://en.wikipedia.org/wiki/Expectation_value_(quantum_mechanics)) of [observables](https://en.wikipedia.org/wiki/Observable#Quantum_mechanics) is an essential task in algorithms in quantum computing, for example, Quantum Neural Networks (QNN), the Variational Quantum Eigensolver (VQE) and the Quantum Approximate Optimization Algorithm (QAOA) [[1]](https://www.nature.com/articles/s42254-021-00348-9). These hybrid quantum-classical algorithms are one of the leading candidates for obtaining quantum advantage in the current era. However, these algorithms require such high precision in estimating the expectation values of observables that make them impractical due to the many sources of noisy in current hardware. 

In this project, we deal with the efficient measurement of the expectation values of observables acting on concentrated states, i.e., when only a small number of amplitudes in a measurement basis are non-negligible. This is accomplished using a hybrid quantum-classical algorithm based on a novel procedure described in [[2]](https://en.wikipedia.org/wiki/Variational_quantum_eigensolver) that includes improvements like reducing the number of CNOT gates employed and the estimated number of parameters. Our algorithm is supported in Qiskit and detailed in this tutorial, which provides the theoretical framework and numerical simulations to evaluate its performance.

# 1. Introduction

The current generation of quantum processors [[3]](https://quantum-journal.org/papers/q-2018-08-06-79/). is characterized by limited numbers of qubits, low gate fidelities, short coherence times, and noise processes that limit circuit depth. In this context, the VQAs is the best hope for obtaining quantum advantage. These algorithms typically utilize quantum computers to measure expectation values of observables for a given state using parametrized [quantum circuits](https://en.wikipedia.org/wiki/Quantum_circuit). However, precisely estimating these expectation values requires numerous [measurement](https://en.wikipedia.org/wiki/Measurement_in_quantum_mechanics) repetitions to reduce statistical fluctuation. Many applications require such high precision, that VQA algorithms are impractical due to the enormous amount of time needed. For instance, quantum chemical calculations require the so-called chemical accuracy of $ 1 \textrm{ kcal/mol } \approx 1.6 \times 10^{-3}$ Hartree, making the [Variational Quantum Eigensolver](https://en.wikipedia.org/wiki/Variational_quantum_eigensolver) (VQE) method non-viable even for a small number of molecules [[4]](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.033154).

In this project, we implement a hybrid quantum-classical algorithm supported on Qiskit to measure the expectation values of observables efficiently. This algorithm is designed to be effective when the state is pure and concentrated, which is a family of quantum states with many interesting practical applications, such as in Quantum Chemistry. The approach is based on the paper [Quantum expectation-value estimation by computational basis sampling](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173) and includes improvements such as a reduction in the number of CNOT gates used and the estimated number of parameters. To explain the algorithm, this tutorial briefly describes the computation of the expectation values of observables. The algorithm's functioning is then described, and its implementation in Qiskit is shown using instructive examples. Moreover, we assess its performance by calculating the resources required to achieve a fixed accuracy. Finally, we illustrate how to implement the VQE method for a molecular Hamiltonian.

## 2.  Measuring expectation values of observables

Let us consider a $n$-qubit system described by the pure state
\begin{align}
|\psi \rangle = \sum_{i=0}^{d-1} c_i |i \rangle,
\end{align}

where $i = i_{n-1} ... i_1 i_0$ is the integer representation of a binary string and $d = 2^n$ is the dimension of the system. The coefficients $c_i$ are complex numbers such that $\sum_{i=0}^{d-1} |c_i|^2 =1$. An observable $O$ acting on $d$ qubits can be expressed as a linear combination of products of Pauli and identity operators on single qubits by

\begin{align}
 O = \sum_{k=1}^M  \alpha_k  P_k,
\end{align}

where $P_k \in \{I, X,Y, Z\}^{\otimes d}$, $\alpha_k\in\mathbb{R}$, and $M$ is the number of the Pauli strings. Therefore, the expectation value of $O$ can be written as

\begin{align}
\langle \psi  | O | \psi \rangle
= \sum_{k=1}^M  \alpha_k \langle \psi  | P_k | \psi \rangle.
\end{align}

In the simplest way to estimate the expectation value $\langle \psi  | O |\psi \rangle$, each
term $\alpha_k \langle \psi  | P_k | \psi \rangle$ is measured on a quantum computer and then
assembled by classical post-processing. However, for many applications, the number of quantities $M$ to be measured is $\mathcal{O}\left(\mathrm{poly}(n)\right)$, making the computation unfeasible if the number of qubits $n$ is not small enough.
There are several studies to reduce the number of measurements to estimate expectation values of observables. Those works, to the best of our knowledge, are driven by: grouping Pauli strings [[4]](https://dx.doi.org/10.1088/1367-2630/18/2/023023), using classical shadows [[5]](https://www.nature.com/articles/s41567-020-0932-7#citeas), and dealing with one part of the quantum state classically [[6]](https://arxiv.org/abs/2106.04755). Despite these efforts, the measurement counts for practical applications are not significantly reduced.

## 3.  Efficient quantum expectation values estimation for concentrated states

### 3.1 Motivation
Under the assumption that $|\psi \rangle$ is a concentrated state, it is defined a reduced state $|\psi_R \rangle$ as

\begin{align}
|\psi_R \rangle = \mathcal{N}_R \sum_{r=1}^{R} c_{i_r} |i_r \rangle,
\end{align}

where $R$ is a fixed number, $|c_{i_1}|^2 \geq |c_{i_2}|^2 \dots \geq |c_{i_R}|^2 \geq \max\{|c_{i}|^2: i \neq i_r \text{ for each } r=1\dots R\}$, and $\mathcal{N}_r = 1/\sqrt{ \sum_{r=1}^{R} |c_{i_r}|^2}$. Therefore, the expectation value of an observable $O$ acting on the system can be approximated by

\begin{equation}
\begin{split}
\langle \psi  | O | \psi \rangle \approx \mathcal{N}_R^2 \langle \psi_R  | O | \psi_R \rangle
= & \,\mathcal{N}_R^2 \sum_{r,r'=1}^R  c_{i_r} c_{i_r'}^* \, \langle i_{r'}  | O | i_r \rangle \\
= & \, \mathcal{N}_R^2 \left\{ \sum_{r=1}^R |c_{i_r}|^2 \langle i_r  | O | i_r \rangle + 2 \sum_{r < r'} \mathrm{Re}(c_{i_r} c_{i_r'}^*) \, \mathrm{Re} \langle i_{r'}  | O | i_r \rangle  -  2 \sum_{ r < r'} \mathrm{Im}(c_{i_r} c_{i_r'}^*) \, \mathrm{Im} \langle i_{r'}  | O | i_r \rangle \right\},
\end{split}
\end{equation}

If $O = O_{n-1} \otimes \dots \otimes O_1 \otimes O_0$ is a tensor product of one-qubit observables, the values of $\langle j |O | k \rangle$ can be efficiently calculated in a classical computer as

\begin{align}
        \langle k |O | j \rangle = \langle k_{n-1} |O_{n-1} | j_{n-1} \rangle \dots  \langle k_1 |O_1 | j_1 \rangle \langle k_0 |O_0 | j_0 \rangle.
\end{align} 

Hence, by an estimation of the coefficients $|c_{i_r}|^2$ , $2\mathrm{Re}(c_{i_r} c_{i_r'}^*)$ and $2\mathrm{Im}(c_{i_r} c_{i_r'}^*)$ we can estimate the expectation value of $O$. For this purpose, we define a [POVM](https://en.wikipedia.org/wiki/POVM) $E$ composed of the elements
\begin{equation}
\begin{split}
P_k = K |k \rangle \langle k |, \quad 
D_{jk} = \frac{K}{2}(|j \rangle  + |k \rangle ) ( \langle  j| + \langle k |), & \quad
A_{jk} =  \frac{K}{2}(|j \rangle  - |k \rangle ) ( \langle  j| - \langle k |), \quad
L_{jk} = \frac{K}{2}(|j \rangle  + i|k \rangle ) ( \langle  j| -i \langle k |), \\
R_{jk} = & \frac{K}{2}(|j \rangle  - i|k \rangle ) ( \langle  j| + i \langle k |), 
\end{split}
\end{equation}

and noticing that
 
\begin{align}
|c_{i}|^2 =   \, \frac{\mathrm{Tr}\left(\rho P_{j} \right)}{K},\qquad
2 \mathrm{Re}(c_j c_k^*) =  \, \frac{\mathrm{Tr}\left(\rho D_{jk} \right) - \mathrm{Tr}\left(\rho A_{jk} \right)}{K},\qquad
-2 \mathrm{Im}(c_j c_k^*) =  \, \frac{\mathrm{Tr}\left(\rho L_{jk} \right) - \mathrm{Tr}\left(\rho R_{jk} \right)}{K},
\end{align}

we approximate $\langle \psi  | O | \psi \rangle$ by measuring the POVM $E$. To implement this POVM, we select $2R(R-1)+1$ basis that contain all the combinations 

$\textbf{Observation:}$ Since concentrate states usually present interactions for a small number of bodies,  there are some quantities $\langle i_{r'}  | O | i_r \rangle = 0 $. Therefore, some coefficients are negligible due to the structure of the quantum state. This reduction of coefficients depend on the number of $X$ and $Y$ gates on the Pauli string, and improves the required measurement resources to compute the approximation, hence improving the estimation of $\langle \psi  | O | \psi \rangle$.


### 3.1 Motivation
Given a state $|\psi \rangle$ of $n$ qubits, the expectation value of an observable $O$ acting on the system is
\begin{equation}
\begin{split}
\langle \psi  | O | \psi \rangle = & \sum_{j,k=1}^d  c_{j} c_{k}^* \, \langle k  | O | j \rangle \\
= &\sum_{j=1}^d |c_{j}|^2 \langle j  | O | j \rangle + 2 \sum_{j < k}^d \mathrm{Re}(c_{j} c_{k}^*) \, \mathrm{Re} \langle k  | O | j \rangle  -  2 \sum_{ j < k}^d \mathrm{Im}(c_j c_k^*) \, \mathrm{Im} \langle k  | O | j \rangle.
\end{split}
\end{equation}

If $O = O_{n-1} \otimes \dots \otimes O_0$ is a tensor product of one-qubit observables, we can classically calculate $ \langle k |O | j \rangle = \langle k_{n-1} |O_{n-1} | j_{n-1} \rangle \dots   \langle k_0 |O_0 | j_0 \rangle$.
Hence, by an estimation of the coefficients $|c_j|^2$ , $2\mathrm{Re}(c_j c_k^*)$ and $2\mathrm{Im}(c_j c_k^*)$ we can estimate the expectation value of $O$. For this purpose, we define a [POVM](https://en.wikipedia.org/wiki/POVM) $E$ composed of the elements

\begin{equation}
\begin{split}
P_k = K |k \rangle \langle k |, \quad 
D_{jk}^\pm = \frac{K}{2}(|j \rangle  \pm |k \rangle ) ( \langle  j| \pm \langle k |), & \quad
L_{jk}^\pm = \frac{K}{2}(|j \rangle  \pm i|k \rangle ) ( \langle  j| \mp i \langle k |), \\
\end{split}
\end{equation}

with $j, k = 0, 1, ..., d-1$, $j < k$ and $K = 1/(2d-1)$ is a normalization term. Noticing that

\begin{align}
|c_{i}|^2 =   \, \frac{\mathrm{Tr}\left(\rho P_{j} \right)}{K},\qquad
2 \mathrm{Re}(c_j c_k^*) =  \, \frac{\mathrm{Tr}\left(\rho D_{jk}^+ \right) - \mathrm{Tr}\left(\rho D_{jk}^- \right)}{K},\qquad
-2 \mathrm{Im}(c_j c_k^*) =  \, \frac{\mathrm{Tr}\left(\rho L_{jk}^+ \right) - \mathrm{Tr}\left(\rho L_{jk}^- \right)}{K},
\end{align}

we approximate $\langle \psi  | O | \psi \rangle$ by measuring the POVM $E$. This POVM can be implemented by sampling uniformly from $2d-1$ basis, which is an exponential number.

We can improve the estimation in two ways: 

1.   If we know that the number of $X$ + $Y$ terms $t$ in a Pauli string $P$ is fixed, then all the terms $\langle k  | P | j \rangle$ such that the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) $\mathrm{dist}_H (k, j) \neq t$ are zero. For example, $\langle 000| XYZ |111\rangle = i\langle 000| 001 \rangle = 0$. Then we only need to measure bases that contain terms of the form $| j \rangle \pm |k \rangle$ and $| j \rangle \pm i|k \rangle$ such that $\mathrm{dist}_H (j, k) = t$. The total number of basis is $K_t = 2\frac{n!}{(n-t)!t!} \ll d$, and we define a new POVM $E_t$ sampling from these bases and with normalization factor $K_t$. These basis require $t-1$ CNOT gates. 

2.   If we know that our state is concentrated, that is, $|\psi \rangle \approx \sum_{r=1}^{R} c_{i_r} |i_r \rangle$, where $R \ll d$ is a fixed number, and $|c_{i_1}|^2 \geq |c_{i_2}|^2 \dots \geq |c_{i_R}|^2 \geq tolerance$, then the only non-vanishing terms will be the $c_{i_r} c_{i_r'}^{*}$. This means that we only have to measure the $K_R \leq 2R(R-1)+1$ bases that contain terms of the form $| i_r \rangle \pm |i_r' \rangle$ and $| i_r \rangle \pm i| i_r' \rangle$ and define a POVM $E_R$ that contains these basis and the normalization factor $K_R$.

The Pauli strings of molecular Hamiltonians mapped to qubits using Jordan Wigner usually contain $0$, $2$ or $4$ $X + Y$ Paulis. Then, we can use (1) with these observables. Also, their ground states are usually highly concentrated. Then, we can use (2). This means that we only have to measure a POVM $E_{Rt}$ that samples from the basis in the intersection of $E_t$ and $E_r$, with a normalization factor $K_{Rt} < K_t, K_r$. Since we are measuring less bases, we can assign more shots to each one of them, reducing the variance and then improving the estimation. 

### 3.2 The method

To implement the above idea we reproduce the following procedure:

1. Prepare the state $|\psi\rangle$ on quantum computers, and measure it in the computational basis $\{| i \rangle : i = 0,1,2^n-1\}$ with a number $L_f$ of shoots.

2. From the outcomes of the measurement we  pick up the most frequent R elements such that $1-|\langle \psi_R | \psi \rangle|^2$ is lower than some previously defined tolerance $\mathrm{Tol}$ ($\approx 10^{-4}$). Next, we sort in descending order of frecuency the selected elements.

3. Determine the number of $X$ and $Y$ gates in the Pauli string to reduce the number of coefficients in the estimation.

4. Measure The POVM $E$ and compute the non-negligible coefficientes $|c_{i_r}|^2$ , $2\,\mathrm{Re}(c_{i_r} c_{i_r'}^*)$ and $2\,\mathrm{Im}(c_{i_r} c_{i_r'}^*)$ as a sample mean of the outcomes.

The described method inherits the performance properties shown in [[2]](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173). First, the error in estimating  $\langle \psi  | O | \psi \rangle$ produced by  approximating  the quantum estate by $| \psi_R\rangle$ is proportional to $\sqrt{1-|\langle \psi_R | \psi \rangle|^2}$. Next, the number of measurements necessary to attain some precision in the standard deviation of the approximation is fewer than that of the alternative techniques. Moreover, the described method improves that reported in [[2]](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173), since the number of CNOT gates employed is not $\mathcal{O}(d)$, but is equal to the number of $X$ and $Y$ gate in the Pauli string.

## 4. Implementation using Qiskit


## 5. Expectation Values of Molecule Hamiltonians

In [49]:
import sys
sys.path.append("../expectation_value")
sys.path.append("../utils_hamiltonian")

from expectationvalue import ExpVal
from utils_hamiltonian import MoleculeHamiltonian, get_number_nbody_terms, get_obs_data
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
import qiskit.quantum_info as qif

import numpy as np
import h5py

The first step is to define a our target molecule and obtain the qubit hamiltonian. bla bla



In [50]:
# The following dictionary gives the molecule geometry needed to obtain
# the hamiltonian, for a given interatomic distance.

molecules = {
    "H2":"H .0 .0 .0; H .0 .0 0.735",
    "LiH":"Li .0 .0 .0; H .0 .0 1.548",
    "BeH2": "Be .0 .0 .0; H .0 .0 -1.7; H .0 .0 1.7",
    "H2O": "H -0.0399 -0.0038 0.0; O 1.5780 0.8540 0.0; H 2.7909 -0.5159 0.0",
    "NH3": "N 0.0 0.0 0.149; H 0.0 0.947 -0.348; H  0.821 -0.474 -0.348; H -0.821 -0.474 -0.348"
}

mol = 'H2'
# mol = 'LiH'
# mol = "H2O"
# mol = "NH3"

Using the tools in utils_hamiltonian and together with qiskit_natures functions, we can easly define our qubit hamiltonian and obtain its exact ground state and ground state energy.

In [51]:
converter = QubitConverter( JordanWignerMapper(), z2symmetry_reduction=None)
molecular_hamiltonian = MoleculeHamiltonian(molecules[mol] , converter=converter, basis='sto3g')

# Qubit hamiltonian
hamiltonian = molecular_hamiltonian.Hamiltonian( freeze_core = False )

driver = molecular_hamiltonian.driver
problem = molecular_hamiltonian.problem

# Number of qubits of the qubit hamiltonian
n_qubits = hamiltonian.num_qubits

# Number of n-bodies interaction
bodies = get_number_nbody_terms(hamiltonian)

# Exact ground state estimation, giving the ground state
# (as a circuit) and the ground state energy
circ_gs, eig_gs = molecular_hamiltonian.ComputeGroundState()

print("Exact Ground State Energy: ", eig_gs)

Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7f8138f34160>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/tempfile.py", line 445, in __del__
    self.close()
  File "/opt/anaconda3/lib/python3.9/tempfile.py", line 441, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/home/dmunoz/Documentos/GitHub/quantum-expectation-value-estimation/Tutorial/tmpvewrrjwk'


Exact Ground State Energy:  -1.857275030202379


Next, we choose the parameters needed to execute our method.

In [52]:
# Total number of resources i.e. ensemble of shots
total_shots = 100000

# Number of shots to measure in the computational basis
r_shots = 1000

# Number of computational basis states retained (Concentration)
r = 2

In [53]:
# We load the parameters of our algorithm
algorithm = ExpVal(n_shots = total_shots-r_shots,
        bodies = bodies, 
        r = r, 
        r_shots = r_shots,
        n_qubits = n_qubits)

# We get the interference terms for a given state, which can be loaded either as a
# quantum circuit or as a vector (numpy array)
algorithm.get_interferences(circ_gs)

# To use the exp_val method, we first need to separete de hamiltonian
# into a list of coefficients and the concatenation of every pauli string
coeffs, obs = get_obs_data(hamiltonian)


# We removed the first pauli string which corresponds to the identity, 
# as its trace with respect to any state is always 1. Therefore,
# we only need to add this coefficient to our expectation value

first_term_coeffs = hamiltonian[0].coeffs[0].real

# We evaluate the expectation value of each string and multiply it for the respective coefficient
exp_val = algorithm.exp_val(obs)
results = np.sum(exp_val*coeffs).real+ first_term_coeffs

In [54]:
print('Calculated expectation value with our algorithm: ', results)
print('Absolute Error: ', np.abs(results-eig_gs))

Calculated expectation value with our algorithm:  -1.8570912360910827
Absolute Error:  0.0001837941112963204


Next, we aim to show that our method is capable of reproducing the same results from [Quantum expectation-value estimation by computational basis sampling](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173) shown in Table 1 and Figure 2, for every molecule in the dictionary `molecules`.

* H2 molecule: total_shots = 300000, r_shots = 1000, r = 2 with 100 repetitions.

* LiH molecule: total_shots = 300000, r_shots = 1000, r = 9 with 100 repetitions.
        
* H2O molecule: total_shots = 4000000, r_shots = 10000, r = 30 with 26 repetitions.
        
* NH3 molecule: total_shots = 10000000, r_shots = 10000, r = 171 with 5 repetitions.

In [45]:
# H2 molecule

h2data = h5py.File('data/h2_data.hdf5', 'r')

h2data_results = h2data["results"][:]
h2data_eigs_gs = h2data['eig_gs']

print('Number of repetitions:', len(h2data_results))

mean_h2data = np.mean(np.abs(h2data_results - h2data_eigs_gs))
std_h2data = np.std(np.abs(h2data_results - h2data_eigs_gs))

print('Mean of the absolute error:', mean_h2data)
print('Standard deviation of the absolute error:', std_h2data)

Number of repetitions: 100
Mean of the absolute error of every repetition: 0.0007403996069078111
Standard deviation of the absolute error of every repetition: 0.0005517153302769676


In [46]:
# LiH molecule

lihdata = h5py.File('data/lih_data.hdf5', 'r')

lihdata_results = lihdata["results"][:]
lihdata_eigs_gs = lihdata['eig_gs']

print('Number of repetitions:', len(lihdata_results))

mean_lihdata = np.mean(np.abs(lihdata_results - lihdata_eigs_gs))
std_lihdata = np.std(np.abs(lihdata_results - lihdata_eigs_gs))

print('Mean of the absolute error:', mean_lihdata)
print('Standard deviation of the absolute error:', std_lihdata)

Number of repetitions: 100
Mean of the absolute error of every repetition: 0.002504397379846406
Standard deviation of the absolute error of every repetition: 0.0018177761250875975


In [47]:
# H2O molecule

h2odata = h5py.File('data/h2o_data.hdf5', 'r')

h2odata_results = h2odata["results"][:]
h2odata_eigs_gs = h2odata['eig_gs']

print('Number of repetitions:', len(h2odata_results))

mean_h2odata = np.mean(np.abs(h2odata_results - h2odata_eigs_gs))
std_h2odata = np.std(np.abs(h2odata_results - h2odata_eigs_gs))

print('Mean of the absolute error:', mean_h2odata)
print('Standard deviation of the absolute error:', std_h2odata)

Number of repetitions: 26
Mean of the absolute error of every repetition: 0.002721312630238952
Standard deviation of the absolute error of every repetition: 0.0017530543271490077


In [48]:
# NH3 molecule

nh3data = h5py.File('data/nh3_data.hdf5', 'r')

nh3data_results = nh3data["results"][:]
nh3data_eigs_gs = nh3data['eig_gs']

print('Number of repetitions:', len(nh3data_results))

mean_nh3data = np.mean(np.abs(nh3data_results - nh3data_eigs_gs))
std_nh3data = np.std(np.abs(nh3data_results - nh3data_eigs_gs))

print('Mean of the absolute error:', mean_h2data)
print('Standard deviation of the absolute error:', std_h2data)

Number of repetitions: 5
Mean of the absolute error of every repetition: 0.0007403996069078111
Standard deviation of the absolute error of every repetition: 0.0005517153302769676


In [ ]:
# To obtain similar results, you could execute the following lines of code,
# choosing beforehand the molecule of interest.

# reps = 100
# results = []

# for i in range(reps):
#     algorithm = ExpVal(n_shots = total_shots-r_shots,
#         bodies = bodies, 
#         r = r, 
#         r_shots = r_shots,
#         n_qubits = n_qubits)
    
#     algorithm.get_interferences(circ_gs)
    
#     coeffs, obs = get_obs_data(hamiltonian)

#     exp_val = algorithm.exp_val(obs)
    
#     # We remove the first pauli string which corresponds to the identity, 
#     # as its trace with respect to any state is always 1. Therefore,
#     # we only need to add this coefficient to our expectation value
    
#     first_term_coeff = hamiltonian[0].coeffs[0].real
    
#     results.append(np.sum(exp_val*coeffs).real + first_term_coeff)

In [30]:
# mol = 'H2'
# mol = 'LiH'
# mol = "H2O"
# mol = "NH3"

converter = QubitConverter( JordanWignerMapper(), z2symmetry_reduction=None)
molecular_hamiltonian = MoleculeHamiltonian(molecules[mol] , converter=converter, basis='sto3g')

# Qubit hamiltonian
hamiltonian = molecular_hamiltonian.Hamiltonian( freeze_core = False )

driver = molecular_hamiltonian.driver
problem = molecular_hamiltonian.problem

# Number of qubits of the qubit hamiltonian
n_qubits = hamiltonian.num_qubits

# Number of n-bodies interaction
bodies = get_number_nbody_terms(hamiltonian)

# Exact ground state estimation, giving the ground state
# (as a circuit) and the ground state energy
circ_gs, eig_gs = molecular_hamiltonian.ComputeGroundState()

print("Exact Ground State Energy: ", eig_gs)

Exact Ground State Energy:  -66.88026272946854


In [31]:
from tqdm import tqdm

reps = 5

# Total number of resources i.e. ensemble of shots
total_shots = 10000000

# Number of shots to measure in the computational basis
r_shots = 10000

# Number of computational basis states retained (Concentration)
r = 30

In [32]:
results = []

for i in tqdm(range(reps)):
    algorithm = ExpVal(n_shots = total_shots-r_shots,
        bodies = bodies, 
        r = r, 
        r_shots = r_shots,
        n_qubits = n_qubits)
    
    algorithm.get_interferences(circ_gs)
    
    coeffs, obs = get_obs_data(hamiltonian)

    exp_val = algorithm.exp_val(obs)
    
    # We remove the first pauli string which corresponds to the identity, 
    # as its trace with respect to any state is always 1. Therefore,
    # we only need to add this coefficient to our expectation value
    
    first_term_coeff = hamiltonian[0].coeffs[0].real
    
    results.append(np.sum(exp_val*coeffs).real + first_term_coeff)

100%|████████████████████████████████████████████| 5/5 [46:45<00:00, 561.19s/it]


In [35]:
print(np.mean(np.abs(results-eig_gs)))
print(np.median(np.abs(results-eig_gs)))
print(np.std(np.abs(results-eig_gs)))

0.001224397053726989
0.0009830372602834814
0.0008589166211297115


In [34]:
# h2_data = h5py.File("h2_data_300000shots_1000rshots_r2_100reps.hdf5", "w")
# h2_data.create_dataset('results', data=results)
# h2_data.create_dataset('eig_gs', data=eig_gs)
# h2_data.close()

# lih_data = h5py.File("lih_data_300000shots_1000rshots_r9_100reps.hdf5", "w")
# lih_data.create_dataset('results', data=results)
# lih_data.create_dataset('eig_gs', data=eig_gs)
# lih_data.close()

# h2o_data = h5py.File("h2o_data_4000000shots_10000rshots_r30_26reps.hdf5", "w")
# h2o_data.create_dataset('results', data=results)
# h2o_data.create_dataset('eig_gs', data=eig_gs)
# h2o_data.close()

nh3_data = h5py.File("nh3_data_10000000shots_1000rshots_r171_Xreps.hdf5", "w")
nh3_data.create_dataset('results', data=results)
nh3_data.create_dataset('eig_gs', data=eig_gs)
nh3_data.close()

In [21]:
# kek = h5py.File('h2_data_300000shots_1000rshots_r2_100reps.hdf5', 'r')
# np.mean(np.abs(kek["results"][:] - kek['eig_gs']))

0.0007403996069078111

Alternatively, we can use qiskit_runtime to calculate the expectation values using the same number of resources, to compare with our method.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

# Number of shots per pauli string
num_paulis = len(hamiltonian)
shots_runtime = total_shots//num_paulis

# Load your qiskit_runtime services
#QiskitRuntimeService.save_account(channel="ibm_quantum", token="1a92dbd3ad8a3a75036e3b0fed7d2a10a0dfef2277960a619c033757aa1aa9bcc8f95922671a2fbd0b05339eea4aae6305bc43b5b2e6d33b161346f9983adf48")
service = QiskitRuntimeService(channel="ibm_quantum")

# Define the options of the Estimator, without error mitigation
# and with the same optimization level as our method (default).

options = Options()
options.optimization_level = 1
options.resilience_level = 0
options.execution.shots = shots_runtime

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    estimator = Estimator(session=session, options = options)
    job = estimator.run(circ_gs, hamiltonian)
    result = job.result()

result_runtime = result.values[0]
np.abs(result_runtime-eig_gs)

## 6. VQE for the H2 Molecule

In [ ]:
class Custom_VQE():
    
    def __init__(self,
                 initial_point,
                 ansatz,
                 optimizer,
                 total_shots,
                 r_shots,
                 r
                ):
        
        self.initial_point = initial_point
        self.ansatz = ansatz
        self.optimizer = optimizer
        
        self.total_shots = total_shots
        self.r_shots = r_shots
        self.r = r
    
    def compute_minimum_eigenvalue(self, hamiltonian):

        def exp_val_vqe(state, hamiltonian):
            
            bodies = get_number_nbody_terms(hamiltonian)
            n_qubits = hamiltonian.num_qubits
            
            coeffs, obs = get_obs_data(hamiltonian)
            
            algorithm = ExpVal(n_shots = self.total_shots - self.r_shots,
                    bodies = bodies, 
                    r = self.r, 
                    r_shots = self.r_shots,
                    n_qubits = n_qubits)
            
            algorithm.get_interferences(state)
            
            first_term_coeff = hamiltonian[0].coeffs[0].real
            
            exp_val_paulis = algorithm.exp_val(obs)
            expectation_value = np.sum(exp_val_paulis*coeffs).real + first_term_coeff
            
            return expectation_value
        
        
        expectation_value = lambda x: exp_val_vqe(self.ansatz.bind_parameters(x), hamiltonian).real
        
        results = optimizer.minimize( expectation_value , self.initial_point )
        return results

In [ ]:
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import SPSA, COBYLA

from qiskit_nature.second_q.circuit.library import UCC, UCCSD, HartreeFock



# ansatz = EfficientSU2(num_qubits=hamiltonian.num_qubits,
#                      reps = 1,
#                      entanglement = "linear",
#                      )

ansatz = UCCSD()
ansatz.num_particles = problem.num_particles
ansatz.num_spatial_orbitals = problem.num_spatial_orbitals
ansatz.qubit_converter = converter

initial_state = HartreeFock()
initial_state.num_particles = problem.num_particles
initial_state.num_spatial_orbitals = problem.num_spatial_orbitals
initial_state.qubit_converter = converter
ansatz.initial_state = initial_state

In [ ]:
initial_point = np.random.randn( ansatz.num_parameters )

In [ ]:
optimizer = COBYLA(maxiter=100)

In [ ]:
from qiskit import Aer, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.utils import QuantumInstance
import qiskit.opflow as of
from functools import reduce

quantum_instance = QuantumInstance( Aer.get_backend("qasm_simulator") , shots=100000)

vqe_try = OurVQE(initial_point=initial_point,
                ansatz = ansatz,
                optimizer = optimizer)

In [ ]:
result_ours = vqe_try.compute_minimum_eigenvalue( hamiltonian )

result_ours.fun

In [ ]:
from qiskit.algorithms import VQE

backend = Aer.get_backend("qasm_simulator")
quantum_instance = QuantumInstance(backend, shots=300000)


vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=quantum_instance)

result = vqe.compute_minimum_eigenvalue(hamiltonian)

result.eigenvalue

## 7. Discussion and outlook

We implement a hybrid quantum-classical algorithm supported by Qiskit to efficiently measure the expectation values of observables. This algorithm is effective when the state is concentrated, a very interesting family of quantum states in practical application, for instance, Quantum Chemistry. The approach presents some improvements to that presented in [[2]](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173) like a reduction in the number of CNOT gates used and a reduction in the estimated number of parameters. The implementation of the algorithm in Qiskit is detailed in this tutorial, where several numerical simulations evaluate its performance.

There are some improvements that could be made on the algorithm implementation:
 
 - Implementing the algorithm in Qiskit is not yet fully polished, as some subroutines are not optimally implemented, causing the program to run slower and consume more resources. 
 
 - The estimator's variance has a closed form, calculated by the program. However, we were not able to perform numerical simulations
 
 - The optimal allocation of the measurement budget was not used because it did not affect the estimation outcome.  We can deduce from this that some minor error caused the program to malfunction.

 - The VQE implementation does not support higher molecules, since the ansatz...



## References 

[1] M. Cerezo, Andrew Arrasmith, Ryan Babbush, Simon C. Benjamin, Suguru Endo, Keisuke Fujii, Jarrod R. McClean, Kosuke Mitarai, Xiao Yuan, Lukasz Cincio & Patrick J. Coles. Variational quantum algorithms. ***Nature Reviews Physics***. 3:625–644, 8 2021. ISSN 2522-5820. [doi: 10.1038/s42254-021-00348-9](https://www.nature.com/articles/s42254-021-00348-9).

[2] Kohda, Masaya and Imai, Ryosuke and Kanno, Keita and Mitarai, Kosuke and Mizukami, Wataru and Nakagawa, Yuya O. Quantum expectation-value estimation by computational basis sampling. ***Phys. Rev. Res***, 4:033173, Sep 2022. [doi:
10.1103/PhysRevResearch.4.033173](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033173).

[3] Gonthier, Jérôme F. and Radin, Maxwell D. and Buda, Corneliu and Doskocil, Eric J. and Abuan, Clena M. and Romero, Jhonathan. Measurements as a roadblock to near-term prac-tical quantum advantage in chemistry: Resource analysis. ***Phys. Rev. Res.***, 4:033154, Aug 2022. [doi: 10.1103/PhysRevResearch.4.033154](https://link.aps.org/doi/10.1103/PhysRevResearch.4.033154).

[4] Jarrod R McClean, Jonathan Romero, Ryan Babbush, and Al ́an Aspuru-Guzik. The theory of variational hybrid
quantum-classical algorithms. ***New Journal of Physics***, 18(2):023023, feb 2016. [doi: 10.1088/1367-2630/18/2/
023023](https://dx.doi.org/10.1088/1367-2630/18/2/023023).

[5] Hsin-Yuan Huang, Richard Kueng, and John Preskill. Predicting many properties of a quantum system from very
few measurements. ***Nature Physics***, 16:1050–1057, 10 2020. ISSN 1745-2473. [doi: 10.1038/s41567-020-0932-7](https://www.nature.com/articles/s41567-020-0932-7).

[6] M. D. Radin and P. Johnson, Classically-boosted variational quantum eigensolver, [arXiv:2106.04755](https://arxiv.org/abs/2106.04755).
